In [ ]:
from huggingface_hub import login,notebook_login
import tqdm as notebook_tqdm

In [ ]:
!git clone https://huggingface.co/h94/IP-Adapter

In [ ]:
!rm -rf IP-Adapter/models/image_encoder
!git clone https://huggingface.co/laion/CLIP-ViT-H-14-laion2B-s32B-b79K IP-Adapter/models/image_encoder


Cloning into 'IP-Adapter/models/image_encoder'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 47 (delta 2), reused 0 (delta 0), pack-reused 39 (from 1)
Unpacking objects: 100% (47/47), 1.08 MiB | 930.00 KiB/s, done.


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import diffusers
print(diffusers.__version__)

0.32.2


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
from google.colab import drive
import os

In [ ]:
# Mount Google Drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from diffusers import DiffusionPipeline
from diffusers import StableDiffusionXLImg2ImgPipeline
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, StableDiffusionInpaintPipelineLegacy, DDIMScheduler, AutoencoderKL
from PIL import Image
import torch


#loading vae path
from diffusers import AutoencoderKL
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)

device="cuda"



In [ ]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

noise_scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
)


In [ ]:
from diffusers import DiffusionPipeline, StableDiffusionXLPipeline, DPMSolverMultistepScheduler
import torch
from IPython.display import Image, display

#loading vae path
from diffusers import AutoencoderKL
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)

device="cuda"

#loading SDXl pipeline
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True).to(device)



lora_path = "/content/drive/MyDrive/sdxl_dreambooth_lora_frog_v2"

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

## add lora weight
pipe.load_lora_weights(lora_path)


prompts = [
    "Tok, A frog, in a colorful yarn world, eating fly, yarn style",
    "The Frog is jumping across yarn hills, whimsical, wide shot, in yarn art style",
    "The Frog is holding a yarn cup, cozy lighting, yarn art style",
    "The Frog is watching stars from a yarn hill at night, dreamy, in yarn art style"
]



Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

## Adding speech bubble

In [ ]:
from PIL import ImageDraw, ImageFont
from PIL import Image, ImageDraw, ImageFont

def resize_image(img, target_size=(384, 384)):
    return img.resize(target_size, Image.Resampling.LANCZOS)


def add_speech_bubble(img, text):
    img = img.copy()
    draw = ImageDraw.Draw(img)

    # Load font with fallback
    try:
        font = ImageFont.truetype("arial.ttf",26)
    except IOError:
        font = ImageFont.load_default()

    padding = 15
    text_bbox = draw.textbbox((0, 0), text, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]

    bubble_width = text_width + 2 * padding
    bubble_height = text_height + 2 * padding

    # Bubble position – top center
    bubble_x = (img.width - bubble_width) // 2
    bubble_y = 10

    # Draw bubble
    draw.rounded_rectangle(
        [(bubble_x, bubble_y), (bubble_x + bubble_width, bubble_y + bubble_height)],
        radius=20,
        fill="white",
        outline="black"
    )

    # Draw text
    draw.text(
        (bubble_x + padding, bubble_y + padding),
        text,
        fill="black",
        font=font
    )

    return img

In [ ]:
generated_images = []

for prompt in prompts:
    image = pipe(
        prompt=prompt,
        negative_prompt="low quality, blurry, deformed, extra limbs, text, watermark, realistic image",
        num_inference_steps=30,
        guidance_scale=7.5,
    ).images[0]
    generated_images.append(image)

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
comic_captions = prompts

In [ ]:

# Resize and add captions
resized_images = [resize_image(img) for img in generated_images]
bubble_images = [add_speech_bubble(img, caption) for img, caption in zip(resized_images, comic_captions)]

# Combine horizontally
panel_width = bubble_images[0].width
panel_height = bubble_images[0].height
comic_panel = Image.new("RGB", (panel_width * 4, panel_height))

for i, img in enumerate(bubble_images):
    comic_panel.paste(img, (i * panel_width, 0))

comic_panel.save("4_panel_comic_FreddyFrog_small.png")

comic_panel.show()
comic_panel.save("/content/drive/MyDrive/Data 298B/SDXL_Dreambooth_lora_character consistency/v5_comic_output/comic_Frog_story_v5.png")


In [ ]:


from PIL import Image

panel_width = generated_images[0].width
panel_height = generated_images[0].height

four_panel = Image.new("RGB", (panel_width * 4, panel_height))

for i, img in enumerate(generated_images):
    four_panel.paste(img, (i * panel_width, 0))

four_panel.show()
four_panel.save()

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]